# Production-Ready RAG Pipeline — Architecture & Steps

This notebook demonstrates a Retrieval-Augmented Generation (RAG) pipeline, covering data ingestion, embeddings, vector storage, and LLM-based querying.


## Table of Contents
1. [Installing Dependencies](#Installing-Dependencies)
2. [API Key Setup](#API-Key-Setup)
3. [Loading PDF Documents](#Loading-PDF-Documents)
4. [Display Loaded Documents](#Display-Loaded-Documents)
5. [Creating a Text Wrapping Function](#Creating-a-Text-Wrapping-Function)
6. [Previewing a Document](#Previewing-a-Document)
7. [Splitting Documents](#Splitting-Documents)
8. [Create embeddings and build FAISS vector store](#Create-embeddings-and-build-FAISS-vector-store)
9. [Setup LLM (Groq)](#Setup-LLM-Groq)
10. [Simple RetrievalQA](#Simple-RetrievalQA)
11. [Conversational RAG with memory](#Conversational-RAG-with-memory)


In [ ]:
# Requirements
%pip install -q langchain langchain_core langchain_community \
    sentence_transformers faiss-cpu unstructured chromadb \
    Cython tiktoken unstructured[local-inference] langchain_groq


## Installing Dependencies

In [ ]:
!pip install -q langchain langchain_core langchain_community sentence_transformers faiss-cpu unstructured chromadb Cython tiktoken unstructured[local-inference] langchain_groq 
# Installs all required packages silently (-q) for your Retrieval-Augmented Generation (RAG) pipeline, including LangChain, FAISS, ChromaDB, and Groq integration.

## API Key Setup

In [3]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Provide your GROQ API TOKEN")

Provide your GROQ API TOKEN··········


## Loading PDF Documents

In [ ]:
# Document Loader
from langchain.document_loaders import UnstructuredPDFLoader  # Imports the UnstructuredPDFLoader to load PDFs.
loader = UnstructuredPDFLoader(r'/content/drive/MyDrive/GenAI vs AgenticAI.pdf') # Creates a loader for the specified PDF file.
documents = loader.load()  # Loads the PDF into memory as documents.

### Display Loaded Documents

In [ ]:
documents   # Displays the contents of the loaded PDF.

[Document(metadata={'source': '/content/drive/MyDrive/GenAI vs AgenticAI.pdf'}, page_content='Generative AI vs. Agentic AI: A Deep Dive into the Future of Intelligent Systems\n\nArtificial Intelligence (AI) is not a monolithic concept, but a vast and evolving landscape of technologies, methodologies, and applications. Among the most significant developments in recent years are Generative AI and Agentic AI—two paradigms that, while closely related, differ fundamentally in their design, function, and implications. As these technologies continue to shape industries, economies, and societies, understanding their distinctions and interplay is critical for anyone engaged in technology, policy, business, or ethics.\n\n1. What is Generative AI?\n\nGenerative AI refers to systems designed to generate new content, data, or information based on learned patterns from existing data. These models use machine learning techniques—particularly deep learning—to create text, images, audio, code, and even

### Creating a Text Wrapping Function

In [ ]:
import textwrap   # Imports Python’s text wrapping module.

def wrap_text_preserve_newlines(text, width=110):    # Defines a function to wrap text to a specified width.
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Previewing a Document

In [7]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='Generative AI vs. Agentic AI: A Deep Dive into the Future of Intelligent Systems

Artificial Intelligence (AI) is not a monolithic concept, but a vast and evolving landscape of technologies,
methodologies, and applications. Among the most significant developments in recent years are Generative AI and
Agentic AI—two paradigms that, while closely related, differ fundamentally in their design, function, and
implications. As these technologies continue to shape industries, economies, and societies, understanding
their distinctions and interplay is critical for anyone engaged in technology, policy, business, or ethics.

1. What is Generative AI?

Generative AI refers to systems designed to generate new content, data, or information based on learned
patterns from existing data. These models use machine learning techniques—particularly deep learning—to create
text, images, audio, code, and even 3D models.

Key Features:

Pattern Recognition: Learns from massive datasets and iden

## Splitting Documents

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter   # Imports a text splitter from LangChain.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [10]:
docs[1]

Document(metadata={'source': '/content/drive/MyDrive/GenAI vs AgenticAI.pdf'}, page_content='Pattern Recognition: Learns from massive datasets and identifies statistical patterns.\n\nContent Creation: Produces human-like output such as essays, paintings, music, or realistic synthetic voices.\n\nNon-Goal-Oriented: It responds to prompts but doesn’t maintain persistent goals or long- term plans.\n\nProminent Examples:\n\nChatGPT: Generates coherent and contextually aware text. \uf0b7 DALL·E: Creates images from text descriptions. \uf0b7 DeepMind’s AlphaCode: Generates code to solve programming problems.\n\nGenerative AI shines in applications requiring creativity, adaptability, and scalability of content creation. However, it typically lacks memory, agency, and persistent intent.\n\n2. What is Agentic AI?')

In [12]:
len(docs)

9

## Create embeddings and build FAISS vector store

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-1664678352.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-1664678352.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hugg

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

## Setup LLM (Groq)

In [21]:
import langchain_groq
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            api_key=os.getenv("GROQ_API_KEY"),
            model="gemma2-9b-it"
        )

## Simple RetrievalQA

In [22]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(GROQ_LLM, chain_type="stuff")

/tmp/ipython-input-2967591166.py:3: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(GROQ_LLM, chain_type="stuff")


### Conversational RAG with memory

In [75]:
query = "HOW IS INDIAN ECONOMY"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'This document provides a description of Generative AI and Agentic AI. However, it does not contain information about the Indian economy. \n\n\nI cannot answer your question about the Indian economy as it is not covered in the provided context. \n'

In [78]:
query = " What are the Risks and Challenges for Generative AI "
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Here are the risks and challenges for Generative AI, as outlined in the text:\n\n* **Misinformation and Deepfakes:** Generative AI can create convincing fake content, which can be used to spread misinformation and manipulate people.\n* **Bias and Harm:** These models can learn and amplify existing biases present in the data they are trained on, leading to unfair or harmful outputs.\n* **Intellectual Property:** There are legal questions surrounding the ownership of intellectual property generated by AI models, especially concerning the training data used. \n\n\nLet me know if you have any other questions. \n'

In [39]:
query = "Who is the Prime Minister of India"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'This document does not contain the answer to who the Prime Minister of India is.  \n'

In [45]:
query = "What are the Examples and Use Cases of Agentic AI"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Here are some examples and use cases of Agentic AI, according to the text provided:\n\n* **Personal Productivity:** Scheduling meetings, drafting reports, managing tasks\n* **Customer Support:** Multi-turn ticket resolution, contextual responses\n* **Research and Analysis:** Autonomous data gathering, hypothesis testing\n* **Automation:** Software agents that use APIs to perform tasks (e.g., booking travel) \n\n\nLet me know if you have any other questions! \n'

In [81]:
query = "List out the Generative AI Strengths and Agentic AI Strengths"

# Retrieve similar documents
docs = db.similarity_search(query)

# Run the chain
response = chain.run(input_documents=docs, question=query)

# Neatly print everything
print("\n" + "="*60)
print("Query:")
print(query)

print("\n Response:")
print(response)
print("="*60 + "\n")



Query:
List out the Generative AI Strengths and Agentic AI Strengths

 Response:
Here are the strengths of each type of AI, as listed in the text:

**Generative AI Strengths:**

* Mimics human expression convincingly 
* Efficiently creates media content at scale
* Aids brainstorming and ideation
* Enhances creative workflows

**Agentic AI Strengths:**

* Automates complex workflows
* Makes strategic decisions
* Learns and adapts through interaction
* Integrates multiple tools and data sources 


Let me know if you have any other questions. 


